In [1]:
import random
from pathlib import Path
import spacy
import numpy as np
import pandas as pd
import os
import re

In [45]:
df_train=pd.read_csv('./Defect Logs/Data/NER_Train.csv',encoding='latin1')
df_train=df_train.apply(lambda x: x.astype(str).str.lower())

df_test=pd.read_csv('./Defect Logs/Data/NER_Test.csv',encoding='latin1')
df_test=df_test.apply(lambda x: x.astype(str).str.lower())

labels=df_test["Entities"].unique()
df_train= df_train.groupby('Description').agg({ 'Entities': ','.join, 'Annotations': ','.join }).reset_index()
df_test= df_test.groupby('Description').agg({ 'Entities': ','.join, 'Annotations': ','.join }).reset_index()

df_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)

In [46]:
df_train

,Description,Entities,Annotations
0,*below logic should be corrected in bteq_pi_rd...,data integrity,count mismatch
1,affinty derivation from pi_afnty to work_atrbn...,functional issue,should consider
2,consumer_ids with no prior provider_preference...,data integrity,not loaded
3,duplicate in target -\nsel a.* \nfrom t16_etl_...,duplicate issue,duplicate
4,duplicate member months are generated in pi_at...,duplicate issue,duplicate
5,duplicate records found -\n\n*select* * *from*...,duplicate issue,duplicate
6,duplicate records in target table\nselect cons...,duplicate issue,duplicate
7,duplicate spans are loaded in work_atrbn_consu...,duplicate issue,duplicate
8,"essentials - medicare - nm member months gone,...",data integrity,lower than expected
9,filter control job failed with the below error...,syntax issue,job failed


In [42]:
df_test

,Description,Entities,Annotations
0,# complete overlap records in pi_atrbn_vbr_mm_...,duplicate issue,overlap records
1,duplicate status updates in pi_atrbn_rule_run_...,duplicate issue,duplicate
2,failure statement: \n\nfailure 3604 cannot pla...,syntax issue,failure
3,filter control job failed in funding exclusion...,syntax issue,job failed
4,in the age filters in retro provider pairing -...,date issue,age range
5,"in work_atrbn_rule_eval_pp, provider pairing d...",functional issue,no check for
6,nr1 provider_preference is not updated in work...,data integrity,not updated
7,vbr tin-npi substitution rule is failing,syntax issue,rule is failing
8,"load to pi_atrbn_rqst_dtl is failing, no more...",spool issue,spool space


In [20]:
#df_test = df_test.drop(6)

In [47]:
df_test

,Description,Entities,Annotations
0,# complete overlap records in pi_atrbn_vbr_mm_...,duplicate issue,overlap records
1,duplicate status updates in pi_atrbn_rule_run_...,duplicate issue,duplicate
2,failure statement: \n\nfailure 3604 cannot pla...,syntax issue,failure
3,filter control job failed in funding exclusion...,syntax issue,job failed
4,in the age filters in retro provider pairing -...,date issue,age range
5,"in work_atrbn_rule_eval_pp, provider pairing d...",functional issue,no check for
6,nr1 provider_preference is not updated in work...,data integrity,not updated
7,vbr tin-npi substitution rule is failing,syntax issue,rule is failing
8,"load to pi_atrbn_rqst_dtl is failing, no more...",spool issue,spool space


In [48]:
TRAIN_DATA=[]
for i in range(len(df_train.index)):
        annotations_extracted=str(df_train.loc[i,'Annotations']).split(",")
        entities_extracted=str(df_train.loc[i,'Entities']).split(",")
        data=[]
        #print(i)
        for j in range(len(annotations_extracted)):
            annotation=annotations_extracted[j].strip()
            print(annotation)
            a = re.search(r'\b(%s)\b'%annotation, df_train.loc[i,'Description'])
            #print(a)
            start_index=a.start()
            end_index=int(start_index)+len(annotation)
            data.append((start_index,end_index,entities_extracted[j]))
        TRAIN_DATA.append((df_train.loc[i,'Description'].strip(),{'entities':data}))

count mismatch
should consider
not loaded
duplicate
duplicate
duplicate
duplicate
duplicate
lower than expected
job failed
job failing
spool space
process is not executing
age
filter is failing
afnty_trmntn_rsn_cd
query should include
overlapping
manual mct requests
not getting loaded
rule failing
rule failing
drop
not updated
overlap
overlapping
duplicate
multiple records
overlapping
not populating correctly
dates
not getting derived properly
atrbn_trmntn_dt is not derived
syntax error
duplicates
join condition
overlaps
records are not getting pulled
records deleted
script failed
duplicates
subject area name
matching with source
versioning is not happening
member should be dropped
mismatch between the records
not occuring as expected
not truncate
unable to load
overlapping
terminated in the target


In [27]:
#a = re.search(r'\b(%s)\b'%annotation, df_train.loc[i,'Description'])

In [44]:
#df_train.loc[i,'Description']

"please find below the excerpts from vishwa's email. \n\nall such overlapping records are because the member is associated to multiple pi_pgm_ids for the same time period. it is because of an issue in the code in prod exclusion tech rule\xa0 \n\nwe are ignoring those programs which have pgm_lob_efctv_dt greater than the prod_cf_xgrpg_efctv_dt which is incorrect. it is currently only impacting one program id which is incorrectly associated *nyclr0101.* \n\n\xa0**\xa0also, i have updated the code to use pgm_lob_psl dates instead of pgm_lob dates to check for overlaps and split the spans (updated case when) \n\n\xa0inner join prod_cf_xgrpg pxg \n\non pex.prod_cf_xgrp_id = pxg.prod_cf_xgrp_id \n\nand pxg.prod_cf_xgrpg_efctv_dt<=plob.pgm_lob_trmntn_dt \n\nand plob.pgm_lob_efctv_dt<=pxg.prod_cf_xgrpg_efctv_dt \n\n\xa0 \n\n*below is the fix for that* \n\n\xa0 \n\ninner join prod_cf_xgrpg pxg \n\non pex.prod_cf_xgrp_id = pxg.prod_cf_xgrp_id \n\nand pxg.prod_cf_xgrpg_efctv_dt<=plobpsl.pgm_lob_p

In [49]:
 TEST_DATA=[]
for i in range(len(df_test.index)):
        annotations_extracted=str(df_test.loc[i,'Annotations']).split(",")
        entities_extracted=str(df_test.loc[i,'Description']).split(",")
        data=[]
        for j in range(len(annotations_extracted)):
            annotation=annotations_extracted[j].strip()
            print(annotation)
            a = re.search(r'\b(%s)\b'%annotation, df_test.loc[i,'Description'])
            #print(a)
            start_index=a.start()
            end_index=int(start_index)+len(annotation)
            data.append((start_index,end_index,entities_extracted[j]))
        TEST_DATA.append((df_test.loc[i,'Description'].strip(),{'entities':data}))

overlap records
duplicate
failure
job failed
age range
no check for
not updated
rule is failing
spool space


In [50]:
TRAIN_DATA

[("*below logic should be corrected in\xa0bteq_pi_rda_vbr_afnty_drvn.sh* count mismatch\n\nfrom\xa0 \nwork_atrbn_consumer_id_span_vbr_pp a \ninner join\xa0 \npi_afnty b\xa0 \non \na.consumer_id = b.consumer_id \nand a.span_strt_dt <= b.afnty_trmntn_dt \nand a.span_end_dt >= b.afnty_efctv_dt\xa0 \n{color:#d04437}*and b.afnty_efctv_dt >= a.span_strt_dt*{color} \n\n{color:#d04437}*--\xa0query to check if the counts for pcp provider pairings match between retro daily and vbr{color}* \n\nselect pi_pgm_id, count(*)\xa0from pi_atrbn_vbr_dtl where afnty_file_type_cd = 'pcp' \nand '2018-01-31' between atrbn_efctv_dt and atrbn_trmntn_dt \nand vrsn_clos_dt = '8888-12-31' and pi_pgm_id like '%esn%' \ngroup by 1; \n\nselect pi_pgm_id, count(*)\xa0from pi_atrbn_r_dtl where afnty_file_type_cd = 'pcp' \nand '2018-01-31' between atrbn_efctv_dt and atrbn_trmntn_dt \nand vrsn_clos_dt = '8888-12-31' and pi_pgm_id like '%esn%' \ngroup by 1;",
  {'entities': [(67, 81, 'data integrity')]}),
 ('affinty deriva

In [51]:
TEST_DATA

[("# complete overlap records in pi_atrbn_vbr_mm_retro \n# need to add cdc logic to account for pi_atrbn_vbr_mm_retro's last runtime",
  {'entities': [(11,
     26,
     "# complete overlap records in pi_atrbn_vbr_mm_retro \n# need to add cdc logic to account for pi_atrbn_vbr_mm_retro's last runtime")]}),
 ('duplicate status updates in pi_atrbn_rule_run_log for the baseline date tweak rule',
  {'entities': [(0,
     9,
     'duplicate status updates in pi_atrbn_rule_run_log for the baseline date tweak rule')]}),
 ('failure statement: \n\nfailure 3604 cannot place a null value in a not null field.',
  {'entities': [(0,
     7,
     'failure statement: \n\nfailure 3604 cannot place a null value in a not null field.')]}),
 ('filter control job failed in funding exclusion bus rule with the following error: \n\nfailure 3604 cannot place a null value in a not null field.',
  {'entities': [(15,
     25,
     'filter control job failed in funding exclusion bus rule with the following error: \n

In [53]:
model=None
n_iter= 50
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


Created blank 'en' model


In [54]:
optimized_dropout_rate=0.5
n_iter=50
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            #print(annotations)
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=optimized_dropout_rate,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print("iteration:"+str(itn+1)+" loss:"+str(losses))

iteration:1 loss:{'ner': 726.5252834494756}
iteration:2 loss:{'ner': 157.65897769864745}
iteration:3 loss:{'ner': 157.59717872197086}
iteration:4 loss:{'ner': 142.80679525172022}
iteration:5 loss:{'ner': 139.69077004841552}
iteration:6 loss:{'ner': 143.35181088380799}
iteration:7 loss:{'ner': 136.19464104197957}
iteration:8 loss:{'ner': 104.47013382661741}
iteration:9 loss:{'ner': 116.21425896354062}
iteration:10 loss:{'ner': 113.73769306711192}
iteration:11 loss:{'ner': 97.7139315220476}
iteration:12 loss:{'ner': 81.38563678683968}
iteration:13 loss:{'ner': 81.38221364311643}
iteration:14 loss:{'ner': 72.65645568613654}
iteration:15 loss:{'ner': 88.99287343517102}
iteration:16 loss:{'ner': 67.63767714154397}
iteration:17 loss:{'ner': 67.81901524231088}
iteration:18 loss:{'ner': 69.11145990167462}
iteration:19 loss:{'ner': 37.53144939705088}
iteration:20 loss:{'ner': 69.02694089984169}
iteration:21 loss:{'ner': 50.09758302414803}
iteration:22 loss:{'ner': 45.54167877155098}
iteration:2

In [56]:
output_dir='./Defect Logs/Data/' 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to Defect Logs\Data


In [59]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
predicted_annotations=[]
predicted_entities=[]
for text, _ in TEST_DATA:
    print(text)
    doc = nlp2(text)
    #print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    result= [(ent.text, ent.label_) for ent in doc.ents]
    print(result)
    predicted_annotations.append([ent.text for ent in doc.ents])
    predicted_entities.append([ent.label_ for ent in doc.ents])

Loading from Defect Logs\Data
# complete overlap records in pi_atrbn_vbr_mm_retro 
# need to add cdc logic to account for pi_atrbn_vbr_mm_retro's last runtime
[('overlap', 'duplicate issue')]
duplicate status updates in pi_atrbn_rule_run_log for the baseline date tweak rule
[('duplicate', 'duplicate issue')]
failure statement: 

failure 3604 cannot place a null value in a not null field.
[]
filter control job failed in funding exclusion bus rule with the following error: 

failure 3604 cannot place a null value in a not null field.
[('job failed', 'syntax issue')]
in the age filters in retro provider pairing - as per the existing logic, while evaluating the members' age range - upper age range number is taken and added to brth_dt to get the upper limit - till when the members' span should be considered. 

correct way of calculating the range is : 

*lower age calculation: birthdate + lower age  (no additional calc needed)* 

*upper age calculation: birthdate + upper age + 1 year - 1 da

In [60]:
df_test['predicted_annotations'] = pd.Series(predicted_annotations, index = df_test.index[:len(predicted_annotations)])
df_test['predicted_entities'] = pd.Series(predicted_entities, index = df_test.index[:len(predicted_entities)]) 

In [61]:
cols = ['predicted_annotations', 'predicted_entities']
for col in cols:
    df_test[col] = df_test[col].map(lambda x: str(x).lstrip("[").rstrip("[")).astype(str)
for col in cols:
    df_test[col] = df_test[col].map(lambda x: str(x).lstrip("]").rstrip("]")).astype(str)

In [62]:
entities_actual=df_test["Entities"].values
annotations_actual=df_test["Annotations"].values
annotations_predicted=df_test['predicted_annotations'].values
entities_predicted=df_test['predicted_entities'].values
#labels=df_test["Entities"].unique()

In [63]:
print(labels)

['data integrity' 'syntax issue' 'functional issue' 'date issue'
 'spool issue' 'duplicate issue']


In [64]:
import csv 
csvfile=open('Confusion_Matrix_NER_NEW1.csv', 'w', newline='')
spamwriter = csv.writer(csvfile)
spamwriter.writerow(['Entity_Name','True Positives','False Positives','True Negatives','False Negatives', 'Precision', 'Recall','F-Score'])

100

In [65]:
for k in range(len(labels)):
    true_positives=0
    false_positives=0
    false_negetives=0
    true_negetives=0
    for i in range(len(entities_actual)):
        pred_ent=str(entities_predicted[i]).split(",")
        act_ent=str(entities_actual[i]).split(",")
        pred_annot=str(annotations_predicted[i]).split(",")
        act_annot=str(annotations_actual[i]).split(",")
        for j in range(len(act_ent)):
            for l in range(len(pred_ent)):
                if (labels[k] in act_ent[j]):
                    if(labels[k] in pred_ent[l]):
                        if act_annot[j] in pred_annot[l]:
                            true_positives=true_positives+1
                        else:
                            false_negetives=false_negetives+1
        
        if (labels[k] in entities_predicted[i])&(labels[k] not in entities_actual[i]):
            false_positives=false_positives+1
        elif (labels[k] not in entities_predicted[i])&(labels[k] in entities_actual[i]):
            false_negetives=false_negetives+1
        else:
            true_negetives=true_negetives+1
            
    if true_positives==0:
        Precision=0
        Recall=0
        Fscore=0
    else:
        Precision=(true_positives)/(true_positives+false_positives)
        Recall=(true_positives)/(true_positives+false_negetives)
        Fscore=2*((Precision*Recall)/(Precision+Recall))
    
    print("Entity_Name:"+str(labels[k]))    
    print("True_Positives:"+str(true_positives)+" False_Positives:"+str(false_positives)+" False_Negatives:"+str(false_negetives)+" True_Negatives:"+str(true_negetives))
    print("Precision:",round(Precision,2))
    spamwriter.writerow([labels[k], true_positives,false_positives,true_negetives,false_negetives, round(Precision,2), round(Recall,2), round(Fscore,2)])
    print("Recall:",round(Recall,2))
    print("Fscore:",round(Fscore,2))
    print("")
csvfile.close()

Entity_Name:data integrity
True_Positives:1 False_Positives:0 False_Negatives:0 True_Negatives:9
Precision: 1.0
Recall: 1.0
Fscore: 1.0

Entity_Name:syntax issue
True_Positives:1 False_Positives:0 False_Negatives:2 True_Negatives:7
Precision: 1.0
Recall: 0.33
Fscore: 0.5

Entity_Name:functional issue
True_Positives:0 False_Positives:0 False_Negatives:1 True_Negatives:8
Precision: 0
Recall: 0
Fscore: 0

Entity_Name:date issue
True_Positives:0 False_Positives:0 False_Negatives:1 True_Negatives:8
Precision: 0
Recall: 0
Fscore: 0

Entity_Name:spool issue
True_Positives:1 False_Positives:0 False_Negatives:0 True_Negatives:9
Precision: 1.0
Recall: 1.0
Fscore: 1.0

Entity_Name:duplicate issue
True_Positives:1 False_Positives:2 False_Negatives:1 True_Negatives:7
Precision: 0.33
Recall: 0.5
Fscore: 0.4

